<a href="https://colab.research.google.com/github/fralfaro/MAT281/blob/main/docs/labs/lab_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAT281 - Laboratorio N°04


**Objetivo**: Aplicar técnicas intermedias y avanzadas de análisis de datos con pandas utilizando un caso real: el Índice de Libertad de Prensa. Este laboratorio incluye operaciones de limpieza, transformación, combinación de datos, y análisis exploratorio usando `merge`, `groupby`, `concat` y otras funciones fundamentales.




**Descripción del Dataset**

El presente conjunto de datos está orientado al análisis del **Índice de Libertad de Prensa**, una métrica internacional que evalúa el nivel de libertad del que gozan periodistas y medios de comunicación en distintos países. Este índice es recopilado anualmente por la organización **Reporteros sin Fronteras**.

La base de datos contempla observaciones por país y año, e incluye tanto el valor del índice como el ranking correspondiente. A menor puntaje en el índice, mayor nivel de libertad de prensa.

**Diccionario de variables**

| Variable     | Clase    | Descripción                                                                          |
| ------------ | -------- | ------------------------------------------------------------------------------------ |
| `codigo_iso` | carácter | Código ISO 3166-1 alfa-3 que representa a cada país.                                 |
| `pais`       | carácter | Nombre oficial del país.                                                             |
| `anio`       | entero   | Año en que se registró la medición del índice.                                       |
| `indice`     | numérico | Valor numérico del Índice de Libertad de Prensa (menor valor indica mayor libertad). |
| `ranking`    | entero   | Posición relativa del país en el ranking mundial de libertad de prensa.              |


**Fuente original y adaptación pedagógica**

* **Fuente original**: [Reporteros sin Fronteras](https://www.rsf-es.org/), recopilado y publicado a través del portal del [Banco Mundial](https://tcdata360.worldbank.org/indicators/h3f86901f?country=BRA&indicator=32416&viz=line_chart&years=2001,2019).
* **Adaptación educativa**: Los archivos han sido modificados intencionalmente para incorporar desafíos técnicos que permiten aplicar los contenidos abordados en clases, tales como limpieza de datos, normalización, detección de duplicados, y combinación de fuentes.


**Descripción de los archivos disponibles**

* **`libertad_prensa_codigo.csv`**: Contiene los pares `codigo_iso` y `pais`. Incluye intencionalmente un código ISO con dos nombres distintos de país para efectos de limpieza y validación de datos.

* **`libertad_prensa_01.csv`**: Contiene registros de los años **anteriores a 2010**. Incluye las variables `PAIS`, `ANIO`, `INDICE`, y `RANKING` con nombres de columna en **mayúsculas**.

* **`libertad_prensa_02.csv`**: Contiene registros de los años **desde 2010 en adelante**. Estructura similar al archivo anterior, con nombres de columna también en **mayúsculas**.





In [5]:
import numpy as np
import pandas as pd

# lectura de datos
archivos_anio = [
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_01.csv',
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_02.csv'
 ]
df_codigos = pd.read_csv('https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_codigo.csv')



### 1. Consolidación y limpieza de datos

A partir de los archivos disponibles, realice los siguientes pasos:

**a)** Cree un DataFrame llamado `df_anio` que consolide la información proveniente de los archivos **`libertad_prensa_01.csv`** y **`libertad_prensa_02.csv`**, correspondientes a distintas ventanas de tiempo. Recuerde que ambos archivos tienen nombres de columnas en mayúscula, por lo que debe normalizarlas a **minúscula** para asegurar consistencia.

**b)** Explore el archivo **`libertad_prensa_codigo.csv`** e identifique el código ISO que aparece asociado a dos nombres de país distintos. Elimine el registro que corresponda a un valor incorrecto o inconsistente, conservando solo el que considere válido.

**c)** Una vez preparados los archivos, cree un nuevo DataFrame llamado `df` que combine `df_anio` con `df_codigos`, utilizando la columna `codigo_iso` como clave. Asegúrese de realizar una unión que conserve únicamente los registros que tengan coincidencia en ambas fuentes.

> **Sugerencia**:
>
> * Para unir los archivos por filas (años), utilice la función `pd.concat([...])`.
> * Para combinar información por columnas (variables), utilice `pd.merge(...)` especificando `on='codigo_iso'`.



In [6]:
import pandas as pd

df_anio_01 = pd.read_csv(archivos_anio[0]).rename(columns=str.lower)
df_anio_02 = pd.read_csv(archivos_anio[1]).rename(columns=str.lower)
df_anio = pd.concat([df_anio_01, df_anio_02], ignore_index=True)

duplicate_iso = df_codigos[df_codigos.duplicated(subset=['codigo_iso'], keep=False)]
display(duplicate_iso)


df_codigos_cleaned = df_codigos[df_codigos['pais'] != 'Congo (Rep. Dem.)'].copy()
display(df_codigos_cleaned.head())



df = pd.merge(df_anio, df_codigos_cleaned, on='codigo_iso', how='inner')

display(df.head())
display(df.info())

,codigo_iso,pais
179,ZWE,Zimbabue
180,ZWE,malo


,codigo_iso,pais
0,AFG,Afghanistán
1,AGO,Angola
2,ALB,Albania
3,AND,Andorra
4,ARE,Emiratos Árabes Unidos


,codigo_iso,anio,indice,ranking,pais
0,AFG,2001,35.5,59.0,Afghanistán
1,AGO,2001,30.2,50.0,Angola
2,ALB,2001,NaN,NaN,Albania
3,AND,2001,NaN,NaN,Andorra
4,ARE,2001,NaN,NaN,Emiratos Árabes Unidos


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3077 entries, 0 to 3076
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   codigo_iso  3077 non-null   object 
 1   anio        3077 non-null   int64  
 2   indice      2680 non-null   float64
 3   ranking     2854 non-null   float64
 4   pais        3077 non-null   object 
dtypes: float64(2), int64(1), object(2)
memory usage: 120.3+ KB


None



### 2. Exploración inicial del conjunto de datos

Una vez que hayas consolidado el DataFrame final `df`, realiza un análisis exploratorio básico respondiendo las siguientes preguntas:

#### **Estructura del DataFrame**

* ¿Cuántas **filas (observaciones)** contiene el conjunto de datos?
* ¿Cuántas **columnas** tiene el DataFrame?
* ¿Cuáles son los **nombres de las columnas**?
* ¿Qué **tipo de datos** tiene cada columna?
* ¿Hay columnas con un tipo de dato inesperado (por ejemplo, fechas como strings)?

#### **Resumen estadístico**

* Genera un resumen estadístico del conjunto de datos con `.describe()`.
  ¿Qué observas sobre los valores de `indice` y `ranking`?
* ¿Qué valores mínimo, máximo y promedio tiene la columna `indice`?
* ¿Qué países presentan los valores extremos en `indice` y `ranking`?

#### **Datos faltantes**

* ¿Cuántos valores nulos hay en cada columna?
* ¿Qué proporción de observaciones tienen valores faltantes?
* ¿Hay columnas con más del 30% de datos faltantes?

#### **Unicidad y duplicados**

* ¿Cuántos países distintos (`pais`) hay en el DataFrame?
* ¿Cuántos años distintos (`anio`) hay representados?
* ¿Existen filas duplicadas (exactamente iguales)? ¿Cuántas?

#### **Validación cruzada de columnas**

* ¿Hay inconsistencias entre el país (`pais`) y su código (`codigo_iso`)?
  (por ejemplo, un mismo código ISO asociado a más de un país)

> **Sugerencia**: Apoya tu análisis con funciones como `.info()`, `.nunique()`, `.isnull().sum()`, `.duplicated()`, `.value_counts()`, entre otras.



    

In [7]:
# Estructura del DataFrame
print("Estructura del DataFrame:")
print(f"Número de filas: {df.shape[0]}")
print(f"Número de columnas: {df.shape[1]}")
print(f"Nombres de las columnas: {df.columns.tolist()}")
print("\nTipo de datos de cada columna:")
display(df.dtypes)

# Resumen estadístico
print("\nResumen estadístico:")
display(df.describe())

print(f"\nValor mínimo de 'indice': {df['indice'].min()}")
print(f"Valor máximo de 'indice': {df['indice'].max()}")
print(f"Valor promedio de 'indice': {df['indice'].mean()}")

# Países con valores extremos en 'indice' y 'ranking'
pais_min_indice = df.loc[df['indice'].idxmin()]['pais']
pais_max_indice = df.loc[df['indice'].idxmax()]['pais']
pais_min_ranking = df.loc[df['ranking'].idxmin()]['pais']
pais_max_ranking = df.loc[df['ranking'].idxmax()]['pais']

print(f"\nPaís con el valor mínimo de 'indice': {pais_min_indice}")
print(f"País con el valor máximo de 'indice': {pais_max_indice}")
print(f"País con el valor mínimo de 'ranking': {pais_min_ranking}")
print(f"País con el valor máximo de 'ranking': {pais_max_ranking}")

# Datos faltantes
print("\nValores nulos por columna:")
display(df.isnull().sum())

print(f"\nProporción de observaciones con valores faltantes: {df.isnull().sum().sum() / len(df) * 100:.2f}%")

print("\nColumnas con más del 30% de datos faltantes:")
for col in df.columns:
    if df[col].isnull().sum() / len(df) > 0.3:
        print(col)

# Unicidad y duplicados
print(f"\nNúmero de países distintos: {df['pais'].nunique()}")
print(f"Número de años distintos: {df['anio'].nunique()}")
print(f"Número de filas duplicadas: {df.duplicated().sum()}")

# Validación cruzada de columnas
print("\nValidación cruzada de codigo_iso y pais:")
display(df.groupby('codigo_iso')['pais'].nunique().sort_values(ascending=False).head())

Estructura del DataFrame:
Número de filas: 3077
Número de columnas: 5
Nombres de las columnas: ['codigo_iso', 'anio', 'indice', 'ranking', 'pais']

Tipo de datos de cada columna:


,0
codigo_iso,object
anio,int64
indice,float64
ranking,float64
pais,object



Resumen estadístico:


,anio,indice,ranking
count,3077.000000,2680.000000,2854.000000
mean,2009.941176,204.836847,475.715137
std,5.786018,2687.491660,6455.679267
min,2001.000000,0.000000,1.000000
25%,2005.000000,15.490000,35.000000
50%,2009.000000,28.150000,71.000000
75%,2015.000000,41.447500,110.000000
max,2019.000000,64536.000000,121056.000000



Valor mínimo de 'indice': 0.0
Valor máximo de 'indice': 64536.0
Valor promedio de 'indice': 204.83684701492538

País con el valor mínimo de 'indice': Dinamarca
País con el valor máximo de 'indice': Kosovo
País con el valor mínimo de 'ranking': Finlandia
País con el valor máximo de 'ranking': Kosovo

Valores nulos por columna:


,0
codigo_iso,0
anio,0
indice,397
ranking,223
pais,0



Proporción de observaciones con valores faltantes: 20.15%

Columnas con más del 30% de datos faltantes:

Número de países distintos: 180
Número de años distintos: 17
Número de filas duplicadas: 0

Validación cruzada de codigo_iso y pais:


,pais
codigo_iso,
ZWE,2
AFG,1
ALB,1
AGO,1
ARE,1





### 3. Comparación regional: países latinoamericanos

En esta sección se busca identificar cuáles son los países de América Latina que han presentado los valores extremos del **Índice de Libertad de Prensa** en cada año observado.

> Recuerda que un menor puntaje en `indice` implica mayor libertad de prensa.

#### **Tareas:**

**a)** Utilizando un ciclo `for`, recorre cada año del conjunto de datos filtrado por países latinoamericanos, y determina para cada año:

* El país con el menor valor de `indice` (mayor libertad de prensa).
* El país con el mayor valor de `indice` (menor libertad de prensa).

**b)** Resuelve la misma tarea del punto anterior utilizando un enfoque vectorizado con `groupby`, sin usar ciclos explícitos.



#### **Lista de países latinoamericanos considerada:**

```python
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
           'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
           'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
           'USA', 'VEN']
```

> Puedes usar esta lista para filtrar el DataFrame final por la columna `codigo_iso`.



In [9]:
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
       'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
       'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
       'USA', 'VEN']

df_america = df[df['codigo_iso'].isin(america)].copy()

print("Análisis por año (usando ciclo for):")
for anio in df_america['anio'].unique():
    df_anio_actual = df_america[df_america['anio'] == anio]

    if not df_anio_actual.empty and df_anio_actual['indice'].notna().any():
        pais_menor_indice = df_anio_actual.loc[df_anio_actual['indice'].idxmin()]
        pais_mayor_indice = df_anio_actual.loc[df_anio_actual['indice'].idxmax()]

        print(f"\nAño {anio}:")
        print(f"  País con menor índice (mayor libertad): {pais_menor_indice['pais']} (Índice: {pais_menor_indice['indice']})")
        print(f"  País con mayor índice (menor libertad): {pais_mayor_indice['pais']} (Índice: {pais_mayor_indice['indice']})")
    else:
        print(f"\nAño {anio}: No hay datos de índice disponibles.")

print("\nAnálisis por año (usando groupby):")

min_indice_per_year = df_america.loc[df_america.groupby('anio')['indice'].idxmin()]
print("\nPaís con menor índice por año (groupby):")
display(min_indice_per_year[['anio', 'pais', 'indice']])

max_indice_per_year = df_america.loc[df_america.groupby('anio')['indice'].idxmax()]
print("\nPaís con mayor índice por año (groupby):")
display(max_indice_per_year[['anio', 'pais', 'indice']])

Análisis por año (usando ciclo for):

Año 2001:
  País con menor índice (mayor libertad): Canadá (Índice: 0.8)
  País con mayor índice (menor libertad): Cuba (Índice: 90.3)

Año 2002:
  País con menor índice (mayor libertad): Trinidad y Tobago (Índice: 1.0)
  País con mayor índice (menor libertad): Cuba (Índice: 97.83)

Año 2003:
  País con menor índice (mayor libertad): Trinidad y Tobago (Índice: 2.0)
  País con mayor índice (menor libertad): Argentina (Índice: 35826.0)

Año 2004:
  País con menor índice (mayor libertad): Trinidad y Tobago (Índice: 2.0)
  País con mayor índice (menor libertad): Cuba (Índice: 87.0)

Año 2005:
  País con menor índice (mayor libertad): Bolivia (Índice: 4.5)
  País con mayor índice (menor libertad): Cuba (Índice: 95.0)

Año 2006:
  País con menor índice (mayor libertad): Canadá (Índice: 4.88)
  País con mayor índice (menor libertad): Cuba (Índice: 96.17)

Año 2007:
  País con menor índice (mayor libertad): Canadá (Índice: 3.33)
  País con mayor índice (me

,anio,pais,indice
27,2001,Canadá,0.80
344,2002,Trinidad y Tobago,1.00
525,2003,Trinidad y Tobago,2.00
706,2004,Trinidad y Tobago,2.00
745,2005,Bolivia,4.50
932,2006,Canadá,4.88
1113,2007,Canadá,3.33
1294,2008,Canadá,3.70
1619,2009,Estados Unidos,6.75
1710,2012,Jamaica,9.88



País con mayor índice por año (groupby):


,anio,pais,indice
39,2001,Cuba,90.30
220,2002,Cuba,97.83
367,2003,Argentina,35826.00
582,2004,Cuba,87.00
763,2005,Cuba,95.00
944,2006,Cuba,96.17
1125,2007,Cuba,88.33
1306,2008,Cuba,94.00
1487,2009,Cuba,78.00
1668,2012,Cuba,71.64


### 4. Análisis anual del índice por país

En esta sección se busca analizar la evolución del **índice máximo** de libertad de prensa alcanzado por cada país a lo largo del tiempo.

#### **Tarea principal:**

* Construye una tabla dinámica (`pivot_table`) donde las **filas** correspondan a los países, las **columnas** a los años (`anio`) y los **valores** sean el `indice` máximo alcanzado por cada país en ese año.
* Asegúrate de reemplazar los valores nulos resultantes con `0`.

> **Hint**: Puedes utilizar el parámetro `fill_value=0` en `pd.pivot_table(...)`.



#### **Preguntas adicionales:**

**a)** ¿Qué país tiene el mayor valor de `indice` en toda la tabla resultante? ¿Y cuál tiene el menor (distinto de cero)?
**b)** ¿Qué años presentan en promedio los valores de `indice` más altos? ¿Y los más bajos?

> (Pista: usa `.mean(axis=0)` sobre la tabla pivot)

**c)** ¿Qué país muestra mayor **variabilidad** (diferencia entre su máximo y mínimo `indice` a lo largo del tiempo)?

> (Pista: aplica `.max(axis=1) - .min(axis=1)`)

**d)** ¿Existen países con índice constante a lo largo de todos los años registrados? ¿Cuáles?

**e)** ¿Qué países no tienen ningún dato (es decir, quedaron con todos los valores igual a 0)? ¿Podrías explicar por qué?





In [12]:
pivot_indice = pd.pivot_table(df,
                              values='indice',
                              index='pais',
                              columns='anio',
                              aggfunc='max',
                              fill_value=0)

display(pivot_indice.head())

# a)
max_indice_total = pivot_indice.max().max()
min_indice_total = pivot_indice[pivot_indice > 0].min().min()

pais_max_indice_total = pivot_indice[pivot_indice == max_indice_total].stack().idxmax()[0]
pais_min_indice_total = pivot_indice[pivot_indice == min_indice_total].stack().idxmin()[0]


print(f"\nPaís con el mayor valor de 'indice' en toda la tabla: {pais_max_indice_total} ({max_indice_total})")
print(f"País con el menor valor de 'indice' (distinto de cero) en toda la tabla: {pais_min_indice_total} ({min_indice_total})")

# b)
avg_indice_per_year = pivot_indice.mean(axis=0)

anio_max_avg_indice = avg_indice_per_year.idxmax()
anio_min_avg_indice = avg_indice_per_year.idxmin()

print(f"\nAño con el promedio de 'indice' más alto: {anio_max_avg_indice} ({avg_indice_per_year.max():.2f})")
print(f"Año con el promedio de 'indice' más bajo: {anio_min_avg_indice} ({avg_indice_per_year.min():.2f})")

# c)
indice_variability = pivot_indice.max(axis=1) - pivot_indice.min(axis=1)
pais_mayor_variabilidad = indice_variability.idxmax()

print(f"\nPaís con mayor variabilidad en 'indice': {pais_mayor_variabilidad} ({indice_variability.max():.2f})")

# d)
constant_indice_countries = indice_variability[indice_variability == 0].index.tolist()
# Excluir paises donde todos los valores son 0
countries_with_all_zeros = pivot_indice[(pivot_indice == 0).all(axis=1)].index.tolist()
constant_indice_countries_filtered = [pais for pais in constant_indice_countries if pais not in countries_with_all_zeros]

print(f"\nPaíses con índice constante a lo largo del tiempo (distinto de cero): {constant_indice_countries_filtered}")

# e) paises sin informacion (todos 0)
print(f"\nPaíses que no tienen ningún dato (todos los valores son 0): {countries_with_all_zeros}")

anio,2001,2002,2003,2004,2005,2006,2007,2008,2009,2012,2013,2014,2015,2017,2018,2019
pais,,,,,,,,,,,,,,,,
Afghanistán,35.5,40.17,28.25,39.17,44.25,56.50,59.25,54.25,51.67,37.36,37.07,37.44,37.75,39.46,37.28,36.55
Albania,0.0,6.50,11.50,14.17,18.00,25.50,16.00,21.75,21.50,30.88,29.92,28.77,29.92,29.92,29.49,29.84
Alemania,1.5,1.33,2.00,4.00,5.50,5.75,4.50,3.50,4.25,10.24,10.23,11.47,14.80,14.97,14.39,14.60
Algeria,31.0,33.00,43.50,40.33,40.00,40.50,31.33,49.56,47.33,36.54,36.26,36.63,41.69,42.83,43.13,45.75
Andorra,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6.82,6.82,19.87,19.87,21.03,22.21,24.63



País con el mayor valor de 'indice' en toda la tabla: Kosovo (64536.0)
País con el menor valor de 'indice' (distinto de cero) en toda la tabla: Austria (0.5)

Año con el promedio de 'indice' más alto: 2013 (446.84)
Año con el promedio de 'indice' más bajo: 2001 (20.19)

País con mayor variabilidad en 'indice': Kosovo (64536.00)

Países con índice constante a lo largo del tiempo (distinto de cero): []

Países que no tienen ningún dato (todos los valores son 0): []
